In [ ]:
import pandas as pd
import numpy as np
import re

path = "/Users/sethfried/Fantasy Football/data/player_game_logs/pfr_gamelogs_RB.csv"
df = pd.read_csv(path)

df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
df['Week'] = pd.to_numeric(df['Week'], errors='coerce').astype('Int64')
df = df[df['Week'].notna()].copy()

df = df.rename(columns={'Unnamed: 6_level_0': 'home'})
df['home'] = (
    df['home']
      .fillna('')
      .apply(lambda x: 1 if '@' in str(x) else 0)
      .astype('int')
)

def split_result(res):
    if not isinstance(res, str):
        return pd.Series([0, np.nan, np.nan])
    win = 1 if res.startswith('W') else 0
    m = re.search(r',\s*(\d+)-(\d+)', res)
    if m:
        return pd.Series([win, int(m.group(1)), int(m.group(2))])
    else:
        return pd.Series([win, np.nan, np.nan])

if 'Result' in df.columns:
    df[['win','score_for','score_against']] = df['Result'].apply(split_result)
    df = df.drop(columns=['Result'])

df['season'] = (
    df['Date'].dt.year
      .where(df['Date'].dt.month >= 9,
             df['Date'].dt.year - 1)
      .astype('Int64')
)

df['Did Not Play'] = 0
df['Inactive']     = 0

if 'GS' in df.columns:
    gs_raw = df['GS'].astype('string').fillna('')
    mask_inactive = gs_raw.str.contains('Inactive', na=False)
    mask_dnp      = gs_raw.str.contains('Did Not Play', na=False)
    df.loc[mask_inactive, 'Inactive']     = 1
    df.loc[mask_dnp,      'Did Not Play'] = 1
    df['GS'] = gs_raw.apply(lambda x: 1 if '*' in x else 0).astype('int')
else:
    df['GS'] = 0

df = df.replace({'Did Not Play': 0, 'Inactive': 0})

keep_cols = [
    'Gcar','Week','Date','Team','home','Opp','GS',
    'Passing Cmp','Passing Att','Passing Cmp%','Passing Yds','Passing TD',
    'Passing Int','Passing Y/A','Passing AY/A','Passing Rate','Passing Sk',
    'Rushing Att','Rushing Yds','Rushing TD','Rushing Y/A',
    'Snap Counts OffSnp','Snap Counts Off%','Snap Counts DefSnp','Snap Counts Def%',
    'Snap Counts STSnp','Snap Counts ST%',
    'Player','Position','Year',
    'Receiving Tgt','Receiving Rec','Receiving Yds','Receiving TD',
    'Receiving Ctch%','Receiving Y/Tgt',
    'Fumbles Fmb','Fumbles FL','Fumbles FF','Fumbles FR','Fumbles Yds','Fumbles FRTD',
    'Did Not Play','Inactive','win','season','score_for','score_against'
]
df = df.loc[:, df.columns.intersection(keep_cols)].copy()


for c in ['Team','Opp','Player','Position']:
    if c in df:
        df[c] = df[c].astype('string')

int_cols = [
    'Gcar','Week','GS','win','score_for','score_against','season',
    'Passing Cmp','Passing Att','Passing TD','Passing Int','Passing Sk',
    'Rushing Att','Rushing TD','Snap Counts OffSnp','Snap Counts DefSnp','Snap Counts STSnp',
    'Receiving Tgt','Receiving Rec','Receiving TD',
    'Fumbles Fmb','Fumbles FL','Fumbles FF','Fumbles FR','Fumbles FRTD',
    'Did Not Play','Inactive','Year'
]
for c in int_cols:
    if c in df:
        df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0).astype('Int64')
float_cols = [
    'Passing Cmp%','Passing Yds','Passing Y/A','Passing AY/A','Passing Rate',
    'Rushing Yds','Rushing Y/A',
    'Snap Counts Off%','Snap Counts Def%','Snap Counts ST%',
    'Receiving Yds','Receiving Ctch%','Receiving Y/Tgt',
    'Fumbles Yds'
]
for c in float_cols:
    if c in df:
        df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0.0).astype('float')

pd.set_option('display.max_columns', None)
print(df.dtypes)
print(df.head())



/var/folders/5b/zyfxmg1n27sc10rqxm40tr3r0000gn/T/ipykernel_70493/2894032929.py:7: DtypeWarning: Columns (47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,64,65,66,67,68,70,71,72,73,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,109,110,111,114,115,116,117,120,121,122,123,124,125,126,129,130,131,134,135,140,141,142,143,144,145,146,147,153,157,158,159,161,162,163,164,165,166,167,169,170,172,173,174,175,180,181,182,183,184,185,186,187,190,191,193,194,195,196,197,198,199,200,201,202,204,208,209,210,211,212,213,214,215,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,241,242,243,244,247,248,249,254,255,256,258,259,260,264,266,267,268,269,270,272,273,274,275,276,277,278,281) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
/var/folders/5b/zyfxmg1n27sc10rqxm40tr3r0000gn/T/ipykernel_70493/2894032929.py:61: FutureWarning: Downcasting behavior in `replace` is deprec

Gcar                           Int64
Week                           Int64
Date                  datetime64[ns]
Team                  string[python]
home                           int64
Opp                   string[python]
GS                             Int64
Rushing Att                    Int64
Rushing Yds                  float64
Rushing TD                     Int64
Rushing Y/A                  float64
Receiving Tgt                  Int64
Receiving Rec                  Int64
Receiving Yds                float64
Receiving TD                   Int64
Receiving Ctch%              float64
Receiving Y/Tgt              float64
Fumbles Fmb                    Int64
Fumbles FL                     Int64
Fumbles FF                     Int64
Fumbles FR                     Int64
Fumbles Yds                  float64
Fumbles FRTD                   Int64
Snap Counts OffSnp             Int64
Snap Counts Off%             float64
Snap Counts DefSnp             Int64
Snap Counts Def%             float64
S

In [3]:
import numpy as np
import pandas as pd

# — assume df is your cleaned RB DataFrame from pfr_gamelogs_RB.csv —

# 1. Ensure Date is datetime and recompute season
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['season'] = (
    df['Date'].dt.year
      .where(df['Date'].dt.month >= 9, df['Date'].dt.year - 1)
      .astype('Int64')
)

# 2. Find missing seasons and build placeholder rows
new_rows = []
for player, grp in df.groupby('Player'):
    # only consider real seasons
    seasons = sorted(int(s) for s in grp['season'].dropna().unique())
    if len(seasons) < 2:
        continue
    full_range = range(seasons[0], seasons[-1] + 1)
    missing = [s for s in full_range if s not in seasons]
    for s in missing:
        prev = grp[grp['season'] == s - 1]
        if prev.empty:
            continue
        last_week = int(prev['Week'].max())
        last_gcar = int(prev.sort_values('Week').iloc[-1]['Gcar'])
        pos       = prev.iloc[0]['Position']
        # one dummy row per week
        for w in range(1, last_week + 1):
            row = {col: 0 for col in df.columns}  # default all stats=0
            row.update({
                'Player': player,
                'Position': pos,
                'season': s,
                'Week': w,
                'Gcar': last_gcar,
                'Inactive': 1
            })
            # explicit NaNs for these context fields
            for c in ['Date','Team','home','Opp','win','score_for','score_against']:
                row[c] = np.nan
            new_rows.append(row)

# 3. Append the dummy rows and resort
df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True, sort=False)
df = df.sort_values(['Player','season','Week']).reset_index(drop=True)

# 4. Preview
pd.set_option('display.max_columns', None)
print(df.head(20))


    Gcar  Week       Date Team  home  Opp  GS  Rushing Att  Rushing Yds  \
0      1     1 2020-09-13  GNB   1.0  MIN   0            2         14.0   
1      2     2 2020-09-20  GNB   0.0  DET   0            5         17.0   
2      3     3 2020-09-27  GNB   1.0  NOR   0            0          0.0   
3      4     4 2020-10-05  GNB   0.0  ATL   0            1          3.0   
4      5     6 2020-10-18  GNB   1.0  TAM   0            5         31.0   
5      6     7 2020-10-25  GNB   1.0  HOU   0            5         11.0   
6      7     8 2020-11-01  GNB   0.0  MIN   0            5         21.0   
7      0     9 2020-11-05  GNB   1.0  SFO   0            0          0.0   
8      0    10 2020-11-15  GNB   0.0  JAX   0            0          0.0   
9      0    11 2020-11-22  GNB   1.0  IND   0            0          0.0   
10     0    12 2020-11-29  GNB   0.0  CHI   0            0          0.0   
11     0    13 2020-12-06  GNB   0.0  PHI   0            0          0.0   
12     8    14 2020-12-13

/var/folders/5b/zyfxmg1n27sc10rqxm40tr3r0000gn/T/ipykernel_70493/791735262.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True, sort=False)


In [4]:
# 1) Fill missing home values with 0 and cast to int
df['home'] = df['home'].fillna(0).astype(int)

# 2) Export to CSV
export_path = "/Users/sethfried/Fantasy Football/data/player_game_logs/pfr_gamelogs_RB_cleaned.csv"
df.to_csv(export_path, index=False)

print(f"Exported cleaned RB gamelogs with home as int to: {export_path}")


Exported cleaned RB gamelogs with home as int to: /Users/sethfried/Fantasy Football/data/player_game_logs/pfr_gamelogs_RB_cleaned.csv
